# Explore the Index of Economic Freedom through time since 2013

In [1]:
import pandas as pd

## 2013 data

In [4]:
df_2013 = pd.read_csv("data/index2013_data.csv", index_col=0)

In [5]:
df_2013.head(5)

,CountryID,Country Name,WEBNAME,Region,World Rank,Region Rank,2013 Score,Change in Yearly Score from 2012,Property Rights,Change in Property Rights from 2012,...,Country,Population (Millions),"GDP (Billions, PPP)",GDP Growth Rate (%),5 Year GDP Growth Rate (%),GDP per Capita (PPP),Unemployment (%),Inflation (%),FDI Inflow (Millions),Public Debt (% of GDP)
0,1,Afghanistan,Afghanistan,Asia-Pacific,NaN,NaN,NaN,NaN,NaN,NaN,...,Afghanistan,31.084,29.731,5.737,10.335398,956.448,NaN,11.247,83.411455,12.100
1,2,Albania,Albania,Europe,58.0,27.0,65.2,0.1,30.0,-5.0,...,Albania,3.218,24.910,2.000,4.431419,7741.428,13.5,3.427,1031.362818,58.923
2,3,Algeria,Algeria,Middle East / North Africa,145.0,14.0,49.6,-1.4,30.0,0.0,...,Algeria,35.954,263.661,2.470,2.718594,7333.226,10.0,4.500,2571.000000,9.925
3,4,Angola,Angola,Sub-Saharan Africa,158.0,40.0,47.3,0.6,15.0,-5.0,...,Angola,19.625,115.679,3.404,8.848807,5894.617,NaN,13.500,-5585.529270,30.897
4,5,Argentina,Argentina,South and Central America / Caribbean,160.0,27.0,46.7,-1.3,15.0,-5.0,...,Argentina,40.900,716.419,8.870,6.811779,17516.147,7.2,9.775,7243.148181,44.203


In [6]:
df_2013.tail(5)

,CountryID,Country Name,WEBNAME,Region,World Rank,Region Rank,2013 Score,Change in Yearly Score from 2012,Property Rights,Change in Property Rights from 2012,...,Country,Population (Millions),"GDP (Billions, PPP)",GDP Growth Rate (%),5 Year GDP Growth Rate (%),GDP per Capita (PPP),Unemployment (%),Inflation (%),FDI Inflow (Millions),Public Debt (% of GDP)
180,181,Yemen,Yemen,Middle East / North Africa,113.0,12.0,55.9,0.6,30.0,0.0,...,Yemen,25.130,57.966,-10.480,1.411758,2306.695,NaN,17.610,-712.81,42.522
181,182,Zambia,Zambia,Sub-Saharan Africa,93.0,12.0,58.7,0.4,30.0,0.0,...,Zambia,13.585,21.882,6.565,6.490851,1610.722,14.0,8.659,1981.70,26.072
182,183,Zimbabwe,Zimbabwe,Sub-Saharan Africa,175.0,46.0,28.6,2.3,10.0,0.0,...,Zimbabwe,12.575,6.127,9.319,-0.145682,487.197,95.0,3.470,387.00,70.328
183,184,Somalia,Somalia,Sub-Saharan Africa,NaN,NaN,NaN,NaN,NaN,NaN,...,Somalia,9.100,6.100,2.600,NaN,600.000,NaN,NaN,102.00,NaN
184,185,Kosovo,Kosovo,Europe,NaN,NaN,NaN,NaN,30.0,NaN,...,Kosovo,1.700,11.990,5.200,NaN,7052.000,45.1,7.300,473.00,5.600


In [7]:
df_2013.dtypes

CountryID                                        int64
Country Name                                    object
WEBNAME                                         object
Region                                          object
World Rank                                     float64
Region Rank                                    float64
2013 Score                                     float64
Change in Yearly Score from 2012               float64
Property Rights                                float64
Change in Property Rights from 2012            float64
Freedom from Corruption                        float64
Change in Freedom from Corruption from 2012    float64
Fiscal Freedom                                 float64
Change in Fiscal Freedom from 2012             float64
Gov't Spending                                 float64
Change in Gov't Spending from 2012             float64
Business Freedom                               float64
Change in Business Freedom from 2012           float64
Labor Free